In [ ]:
import mlrose_hiive as RO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [ ]:
seed = 166
# read whitewine dataset
# path = r"C:\Users\Yiwei Zhang\Desktop\CS 7641\Howework 2\winequality.csv"
path = "winequality.csv"
data = pd.read_csv(path)
# Split data into training and test sets
predictors = data[data.columns.difference(["label"])]
target = data[["label"]]

X_train, X_test, y_train, y_test = train_test_split(predictors, target, \
                                                    test_size = 0.2, random_state = 166, stratify=target)

# Normalize feature data
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Gradient Descent - tuning learning rate
# try to tune the parameter of activation: identity, relu, sigmoid, tanh
def gradient_descent(activation, lr, curves, losses):
    nn_model1 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = activation, \
                                     algorithm = 'gradient_descent', max_iters = 3000, \
                                     bias = True, is_classifier = True, learning_rate = lr, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True)
    nn_model1.fit(X_train_scaled, y_train)
    curves.append(nn_model1.fitness_curve)
    losses.append(nn_model1.loss)
    
fitness_curves = list()
best_loss = list()

lr_list = [0.0001, 0.0006, 0.001, 0.0015, 0.002]
for lr in lr_list:
    gradient_descent('sigmoid', lr, fitness_curves, best_loss)


In [ ]:
import copy

for curve in fitness_curves:
    print(curve.shape[0])

for i, curve in enumerate(fitness_curves):
    if len(curve) < 3000:
        n = curve[-1]
        c = [n for _ in range(3000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j] < -1:
            curve[j] = curve[j-1]
    fitness_curves[i] = curve*(-1)

dic = {'Iterations': range(3000),
        '0.0001': fitness_curves[0],
        '0.0005': fitness_curves[1],
        '0.001': fitness_curves[2],
        '0.0015': fitness_curves[3],
        '0.002': fitness_curves[4],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='0.0001', ax=ax)
df.plot(kind='line', x='Iterations', y='0.0005', ax=ax)
df.plot(kind='line', x='Iterations', y='0.001', ax=ax)
df.plot(kind='line', x='Iterations', y='0.0015', ax=ax)
df.plot(kind='line', x='Iterations', y='0.002', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for different learning rates (sigmoid)')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\Gradient Descent.png', dpi=500)
plt.show()

In [ ]:
# gradient descent - test the best parameters
nn_model1 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'gradient_descent', max_iters = 3000, \
                                     bias = True, is_classifier = True, learning_rate = 0.0005, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True)
t0_clock = time.process_time()
nn_model1.fit(X_train_scaled, y_train)
t1_clock = time.process_time()
y_train_pred = nn_model1.predict(X_train_scaled)
y_test_pred = nn_model1.predict(X_test_scaled)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
y_test_recall = recall_score(y_test, y_test_pred)
print(y_train_accuracy, y_test_accuracy, y_test_recall, t1_clock-t0_clock, nn_model1.loss, len(nn_model1.fitness_curve))

In [ ]:
# Randomized Hill Climbing - tuning restarts
def rhc(restart, curves, losses):
    nn_model2 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'random_hill_climb', max_iters = 10000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 500, \
                                     random_state = seed, curve=True, restarts=restart)
    nn_model2.fit(X_train, y_train)
    curves.append(nn_model2.fitness_curve)
    losses.append(nn_model2.loss)
fitness_curves = list()
best_loss = list()

restarts = [0, 1, 2, 3, 4]
for restart in restarts:
    rhc(restart, fitness_curves, best_loss)   

In [ ]:
import copy

for i, curve in enumerate(fitness_curves):
    if len(curve) < 10000:
        n = curve[-1]
        c = [n for _ in range(10000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j, 0] < -1:
            curve[j, 0] = curve[j-1, 0]

dic = {'Iterations': range(10000),
        '0': fitness_curves[0][:, 0],
        '1': fitness_curves[1][:, 0],
        '2': fitness_curves[2][:, 0],
        '3': fitness_curves[3][:, 0],
        '4': fitness_curves[4][:, 0],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='0', ax=ax)
df.plot(kind='line', x='Iterations', y='1', ax=ax)
df.plot(kind='line', x='Iterations', y='2', ax=ax)
df.plot(kind='line', x='Iterations', y='3', ax=ax)
df.plot(kind='line', x='Iterations', y='4', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for different restarts')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\RHC.png', dpi=500)
plt.show()

In [ ]:
# random hill climbing - test the best parameter
nn_model2 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'random_hill_climb', max_iters = 10000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, restarts=2)
t0_clock = time.process_time()
nn_model2.fit(X_train_scaled, y_train)
t1_clock = time.process_time()
y_train_pred = nn_model2.predict(X_train_scaled)
y_test_pred = nn_model2.predict(X_test_scaled)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
y_test_recall = recall_score(y_test, y_test_pred)
print(y_train_accuracy, y_test_accuracy, y_test_recall, t1_clock-t0_clock, nn_model2.loss, len(nn_model2.fitness_curve))

In [ ]:
# Simulated Annealing - tuning Decay schedule
def sa(schedule, curves, losses):
    nn_model3 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'simulated_annealing', max_iters = 3000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, schedule=schedule)
    nn_model3.fit(X_train_scaled, y_train)
    curves.append(nn_model3.fitness_curve)
    losses.append(nn_model3.loss)
fitness_curves = list()
best_loss = list()

schedules = [RO.GeomDecay(), RO.ArithDecay(), RO.ExpDecay()]
for schedule in schedules:
    sa(schedule, fitness_curves, best_loss)   

In [ ]:
import copy

for i, curve in enumerate(fitness_curves):
    if len(curve) < 3000:
        n = curve[-1]
        c = [n for _ in range(3000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j, 0] < -1:
            curve[j, 0] = curve[j-1, 0]

dic = {'Iterations': range(2000),
        'GeomDecay': fitness_curves[0][:2000, 0],
        'ArithDecay': fitness_curves[1][:2000, 0],
        'ExpDecay': fitness_curves[2][:2000, 0],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='GeomDecay', ax=ax)
df.plot(kind='line', x='Iterations', y='ArithDecay', ax=ax)
df.plot(kind='line', x='Iterations', y='ExpDecay', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for different schedule')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\SA1.png', dpi=500)
plt.show()

In [ ]:
# Simulated Annealing - tuning initial temperature
def sa(schedule, curves, losses):
    nn_model3 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'simulated_annealing', max_iters = 3000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, schedule=schedule)
    nn_model3.fit(X_train_scaled, y_train)
    curves.append(nn_model3.fitness_curve)
    losses.append(nn_model3.loss)
fitness_curves = list()
best_loss = list()

temperatures = [0.5, 1, 2, 5, 10]
for temperature in temperatures:
    sa(RO.GeomDecay(init_temp=temperature), fitness_curves, best_loss)   

In [ ]:
import copy

for i, curve in enumerate(fitness_curves):
    if len(curve) < 3000:
        n = curve[-1]
        c = [n for _ in range(3000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j, 0] < -1:
            curve[j, 0] = curve[j-1, 0]

dic = {'Iterations': range(2000),
        '0.5': fitness_curves[0][:2000, 0],
        '1': fitness_curves[1][:2000, 0],
        '2': fitness_curves[2][:2000, 0],
       '5': fitness_curves[3][:2000, 0],
        '10': fitness_curves[4][:2000, 0],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='0.5', ax=ax)
df.plot(kind='line', x='Iterations', y='1', ax=ax)
df.plot(kind='line', x='Iterations', y='2', ax=ax)
df.plot(kind='line', x='Iterations', y='5', ax=ax)
df.plot(kind='line', x='Iterations', y='10', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for different schedule')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\SA2.png', dpi=500)
plt.show()

In [ ]:
# simulated annealing - test the best parameters
nn_model3 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'simulated_annealing', max_iters = 2000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, schedule=RO.GeomDecay(init_temp=0.5))
t0_clock = time.process_time()
nn_model3.fit(X_train_scaled, y_train)
t1_clock = time.process_time()
y_train_pred = nn_model3.predict(X_train_scaled)
y_test_pred = nn_model3.predict(X_test_scaled)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
y_test_recall = recall_score(y_test, y_test_pred)
print(y_train_accuracy, y_test_accuracy, y_test_recall, t1_clock-t0_clock, nn_model3.loss, len(nn_model3.fitness_curve))

In [ ]:
# Generic Algorithm - tuning population size
def ga(pop_size, mut, curves, losses):
    nn_model4 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'genetic_alg', max_iters = 3000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, pop_size=pop_size, mutation_prob=mut)
    nn_model4.fit(X_train_scaled, y_train)
    curves.append(nn_model4.fitness_curve)
    losses.append(nn_model4.loss)
fitness_curves = list()
best_loss = list()

pop_sizes = [10, 20, 50, 80, 100]
for pop_size in pop_sizes:
    ga(pop_size, 0.1, fitness_curves, best_loss) 

In [ ]:
import copy

for curve in fitness_curves:
    print(curve.shape[0])

for i, curve in enumerate(fitness_curves):
    if len(curve) < 3000:
        n = curve[-1]
        c = [n for _ in range(3000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j, 0] < -1:
            curve[j, 0] = curve[j-1, 0]

dic = {'Iterations': range(400, 1000),
        '10': fitness_curves[0][400:1000, 0],
        '20': fitness_curves[1][400:1000, 0],
        '50': fitness_curves[2][400:1000, 0],
       '80': fitness_curves[3][400:1000, 0],
        '100': fitness_curves[4][400:1000, 0],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='10', ax=ax)
df.plot(kind='line', x='Iterations', y='20', ax=ax)
df.plot(kind='line', x='Iterations', y='50', ax=ax)
df.plot(kind='line', x='Iterations', y='80', ax=ax)
df.plot(kind='line', x='Iterations', y='100', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for population size')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\GA1-1.png', dpi=500)
plt.show()

In [ ]:
# Generic Algorithm - tuning mutation probability
def ga(pop_size, mut, curves, losses):
    nn_model4 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'genetic_alg', max_iters = 1000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, pop_size=pop_size, mutation_prob=mut)
    nn_model4.fit(X_train_scaled, y_train)
    curves.append(nn_model4.fitness_curve)
    losses.append(nn_model4.loss)
fitness_curves = list()
best_loss = list()

mutation_probs = [0.1, 0.2, 0.4, 0.6, 0.8]
for prob in mutation_probs:
    ga(20, prob, fitness_curves, best_loss) 

In [ ]:
import copy

for curve in fitness_curves:
    print(curve.shape[0])

for i, curve in enumerate(fitness_curves):
    if len(curve) < 1000:
        n = curve[-1]
        c = [n for _ in range(1000 - len(curve))]
        curve = np.concatenate((curve, c), axis=0)
        fitness_curves[i] = curve
    for j in range(len(curve)):
        if curve[j, 0] < -1:
            curve[j, 0] = curve[j-1, 0]

dic = {'Iterations': range(1000),
        '0.1': fitness_curves[0][:1000, 0],
        '0.2': fitness_curves[1][:1000, 0],
        '0.4': fitness_curves[2][:1000, 0],
       '0.6': fitness_curves[3][:1000, 0],
        '0.8': fitness_curves[4][:1000, 0],
        }    
df = pd.DataFrame(dic, columns=dic.keys())

ax = plt.gca()
df.plot(kind='line', x='Iterations', y='0.1', ax=ax)
df.plot(kind='line', x='Iterations', y='0.2', ax=ax)
df.plot(kind='line', x='Iterations', y='0.4', ax=ax)
df.plot(kind='line', x='Iterations', y='0.6', ax=ax)
df.plot(kind='line', x='Iterations', y='0.8', ax=ax)

plt.xlabel('Iterations')
plt.ylabel('Fitness Scores')
plt.title('Training curves for mutation probability')
plt.grid(True)
plt.savefig(r'C:\Users\Yiwei Zhang\Desktop\GA2.png', dpi=500)
plt.show()

In [ ]:
# genetic algorithm - test the best parameters
nn_model4 = RO.NeuralNetwork(hidden_nodes = [20, 10], activation = 'sigmoid', \
                                     algorithm = 'genetic_alg', max_iters = 1000, \
                                     bias = True, is_classifier = True, learning_rate = 5, \
                                     early_stopping = True, max_attempts = 100, \
                                     random_state = seed, curve=True, pop_size=20, mutation_prob=0.1)
t0_clock = time.process_time()
nn_model4.fit(X_train_scaled, y_train)
t1_clock = time.process_time()
y_train_pred = nn_model4.predict(X_train_scaled)
y_test_pred = nn_model4.predict(X_test_scaled)
y_train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_accuracy = accuracy_score(y_test, y_test_pred)
y_test_recall = recall_score(y_test, y_test_pred)
print(y_train_accuracy, y_test_accuracy, y_test_recall, t1_clock-t0_clock, nn_model4.loss, len(nn_model4.fitness_curve))